In [1]:
from GAME.bin.intertask_mappings import *
from GAME.utils.config import config
from itertools import product
from sklearn.preprocessing import MinMaxScaler

In [2]:
def agg_res(eval_results:dict, state:str=None, action:int=None, strategy:str='average') -> dict:
    agg_results = {}
    state_agg = []
    action_agg = []
    for k, v in eval_results.items():
        key_split = k.split('--')
        key_state = key_split[1].split('-')[1]
        key_action = key_split[0]
        if state and state == key_state:
            state_agg.append(v)
        if action and str(action) == key_action:
            action_agg.append(v)
    if strategy == 'average':
        if state:
            agg_results[state] = np.mean(state_agg)
        if action:
            agg_results[action] = np.mean(action_agg)
    if strategy == 'best':
        if state:
            agg_results[state] = np.max(state_agg)
        if action:
            agg_results[action] = np.max(action_agg)

    return agg_results

In [19]:
# load config data
config_data = config()

src_state_var_names = config_data['3v2_state_names']
src_action_names = config_data['3v2_action_names']
src_action_values = config_data['3v2_action_values']
target_state_var_names = config_data['4v3_state_names']
target_action_names = config_data['4v3_action_names']
target_action_values = config_data['4v3_action_values']

# mapping = IntertaskMapping([0 for _ in range(19)], [0, 1, 2, 2], src_state_var_names, src_action_names, target_state_var_names, target_action_names)
# mapping = IntertaskMapping([11, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12], [0, 1, 2, 2], src_state_var_names, src_action_names, target_state_var_names, target_action_names)

src_data_path = os.path.join(config_data['data_path'], 'keepaway', "keepaway_3v2_transitions.csv")
src_data_df = pd.read_csv(src_data_path, index_col = False)
transformed_df_col_names = config_data['4v3_full_transition_df_col_names']

feature_scaler = MinMaxScaler()
feature_names = ['Current-{}'.format(feature) for feature in src_state_var_names]
feature_scaler.fit(src_data_df[feature_names])
src_data_df[feature_names] = feature_scaler.transform(src_data_df[feature_names])

target_scaler = MinMaxScaler()
target_names = ['Next-{}'.format(feature) for feature in src_state_var_names]
target_scaler.fit(src_data_df[target_names])
src_data_df[target_names] = target_scaler.transform(src_data_df[target_names])

src_data_df['Current-action'] = src_data_df['Current-action'].replace(0, 4)
src_data_df['Current-action'] = src_data_df['Current-action'].replace(1, 0)
src_data_df['Current-action'] = src_data_df['Current-action'].replace(4, 1)

# transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)
# transformed_df_out_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\output\\11022022 Transformed Source Data 3DMC\\transformed_df.csv"
# transformed_df.to_csv(transformed_df_out_path, index = False)

network_folder_path = os.path.join(config_data['pickle_path'], 'neural_nets', "keepaway")
eval_networks = EvaluationNetworks(network_folder_path)
# test_mlp = eval_networks.get_network(0, 'Next_x_position')

transformed_df_current_state_cols = config_data['4v3_current_state_transition_df_col_names']
transformed_df_next_state_cols = config_data['4v3_next_state_transition_df_col_names']

# eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [16]:
state_mapping = [0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11, 12, 12]
# state_mapping = [0 for _ in range(19)]
action_mapping = [0, 1, 2, 2]
mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [17]:
print(parse_mapping_eval_scores(eval_results))

[0.7288566720658226]


In [18]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.8722644839350171}
{'dist(K1,K2)': -0.22939281778175027}
{'dist(K1,K3)': 0.8424341522855624}
{'dist(K1,K4)': 0.9286357208327285}
{'dist(K1,T1)': 0.825480264121766}
{'dist(K1,T2)': 0.8231096644799776}
{'dist(K1,T3)': 0.8276272203865441}
{'dist(K2,C)': 0.6698307880330103}
{'dist(K3,C)': 0.7540449851072928}
{'dist(K4,C)': 0.8598177118728227}
{'dist(T1,C)': 0.922598368239199}
{'dist(T2,C)': 0.9282651407342403}
{'dist(T3,C)': 0.9304682075749454}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.7970593744195932}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.6786768532653569}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': -0.17427110914843588}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.9465057040889643}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.7738532579239163}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.8712687988798752}


In [23]:
# GAME
state_mapping = [0, 11, 5, 0, 4, 3, 4, 9, 6, 6, 7, 9, 9, 8, 2, 4, 11, 10, 9]
# state_mapping = [0 for _ in range(19)]
action_mapping = [1, 1, 1, 1]
mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [24]:
print(parse_mapping_eval_scores(eval_results))

[0.9094175953584726]


In [9]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.8984891033086441}
{'dist(K1,K2)': 0.9832637591162632}
{'dist(K1,K3)': 0.7849590820300703}
{'dist(K1,K4)': 0.954064586502867}
{'dist(K1,T1)': 0.8867525246875843}
{'dist(K1,T2)': 0.9106655510397274}
{'dist(K1,T3)': 0.9243453270534202}
{'dist(K2,C)': 0.8889423560018718}
{'dist(K3,C)': 0.9271890722436928}
{'dist(K4,C)': 0.9442787800076675}
{'dist(T1,C)': 0.8949127149320598}
{'dist(T2,C)': 0.9360098592514783}
{'dist(T3,C)': 0.9502627358934586}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.739350172055359}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.8737771738760037}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': 0.9439320299226585}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.9525089079653637}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.9239104302786245}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.9613201456441616}


In [10]:
# GAME
state_mapping = [0, 12, 9, 1, 4, 4, 4, 1, 11, 9, 9, 9, 9, 4, 9, 9, 9, 11, 9]
# state_mapping = [0 for _ in range(19)]
action_mapping = [0, 0, 0, 0]
mapping = IntertaskMapping(state_mapping, action_mapping, src_state_var_names, src_action_names, target_state_var_names, target_action_names)

transformed_df = transform_source_dataset(src_data_df, mapping, transformed_df_col_names, target_action_values)

eval_results = evaluate_mapping(mapping, transformed_df, eval_networks, transformed_df_current_state_cols, transformed_df_next_state_cols, target_action_values)

In [11]:
print(parse_mapping_eval_scores(eval_results))

[0.9310901661448567]


In [12]:
for state in target_state_var_names:
    print(agg_res(eval_results, state, None, 'average'))

{'dist(K1,C)': 0.9008379489607738}
{'dist(K1,K2)': 0.9561334397595876}
{'dist(K1,K3)': 0.9435489892493593}
{'dist(K1,K4)': 0.9648254132358924}
{'dist(K1,T1)': 0.9429154002937308}
{'dist(K1,T2)': 0.9465724754300435}
{'dist(K1,T3)': 0.9394943913535898}
{'dist(K2,C)': 0.9109813599892254}
{'dist(K3,C)': 0.8942009543309912}
{'dist(K4,C)': 0.9630878849788214}
{'dist(T1,C)': 0.9453022638364438}
{'dist(T2,C)': 0.9392673273512193}
{'dist(T3,C)': 0.9550970606480202}
{'Min(dist(K2,T1),dist(K2,T2),dist(K2,T3))': 0.888458770619969}
{'Min(dist(K3,T1),dist(K3,T2),dist(K3,T3))': 0.9444237029212095}
{'Min(dist(K4,T1),dist(K4,T2),dist(K4,T3))': 0.7335602698333762}
{'Min(ang(K2,K1,T1),ang(K2,K1,T2),ang(K2,K1,T3))': 0.9561325364019457}
{'Min(ang(K3,K1,T1),ang(K3,K1,T2),ang(K3,K1,T3))': 0.9825440605193293}
{'Min(ang(K4,K1,T1),ang(K4,K1,T2),ang(K4,K1,T3))': 0.9833289070387519}
